In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,770 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,962 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
df.createOrReplaceTempView('home_sales')

In [13]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql("""

SELECT YEAR(date) AS Sold_Year, ROUND(AVG(price)) AS AVG_SALE_PRICE FROM home_sales
WHERE bedrooms = 4
GROUP BY Sold_Year
ORDER BY Sold_Year
""").show()

+---------+--------------+
|Sold_Year|AVG_SALE_PRICE|
+---------+--------------+
|     2019|      300264.0|
|     2020|      298354.0|
|     2021|      301819.0|
|     2022|      296364.0|
+---------+--------------+



In [16]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?


spark.sql("""

SELECT date_built, ROUND(AVG(price)) AS AVG_SALE_PRICE FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3

GROUP BY date_built
ORDER BY date_built

""").show()

+----------+--------------+
|date_built|AVG_SALE_PRICE|
+----------+--------------+
|      2010|      292860.0|
|      2011|      291117.0|
|      2012|      293683.0|
|      2013|      295962.0|
|      2014|      290852.0|
|      2015|      288770.0|
|      2016|      290555.0|
|      2017|      292677.0|
+----------+--------------+



In [17]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?


spark.sql("""

SELECT date_built, ROUND(AVG(price)) AS AVG_SALE_PRICE FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built

""").show()

+----------+--------------+
|date_built|AVG_SALE_PRICE|
+----------+--------------+
|      2010|      285010.0|
|      2011|      276554.0|
|      2012|      307540.0|
|      2013|      303677.0|
|      2014|      298265.0|
|      2015|      297610.0|
|      2016|      293965.0|
|      2017|      280318.0|
+----------+--------------+



In [19]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()
spark.sql("""

SELECT view, ROUND(AVG(price)) AS AVG_SALE_PRICE FROM home_sales
GROUP BY view
HAVING AVG_SALE_PRICE >= 350000
ORDER BY view DESC

""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|AVG_SALE_PRICE|
+----+--------------+
| 100|     1026670.0|
|  99|     1061201.0|
|  98|     1053739.0|
|  97|     1129040.0|
|  96|     1017816.0|
|  95|     1054326.0|
|  94|     1033536.0|
|  93|     1026006.0|
|  92|      970403.0|
|  91|     1137373.0|
|  90|     1062654.0|
|  89|     1107839.0|
|  88|     1031719.0|
|  87|     1072285.0|
|  86|     1070444.0|
|  85|     1056337.0|
|  84|     1117233.0|
|  83|     1033966.0|
|  82|     1063498.0|
|  81|     1053473.0|
+----+--------------+
only showing top 20 rows

--- 1.1107370853424072 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales
spark.catalog.cacheTable("home_sales")

# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [37]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql("""

SELECT view, ROUND(AVG(price)) AS AVG_SALE_PRICE FROM home_sales
GROUP BY view
HAVING AVG_SALE_PRICE >= 350000
ORDER BY view DESC

""").show()
print("--- %s seconds ---" % (time.time() - start_time))
# Cached query is much quicker.

+----+--------------+
|view|AVG_SALE_PRICE|
+----+--------------+
| 100|     1026670.0|
|  99|     1061201.0|
|  98|     1053739.0|
|  97|     1129040.0|
|  96|     1017816.0|
|  95|     1054326.0|
|  94|     1033536.0|
|  93|     1026006.0|
|  92|      970403.0|
|  91|     1137373.0|
|  90|     1062654.0|
|  89|     1107839.0|
|  88|     1031719.0|
|  87|     1072285.0|
|  86|     1070444.0|
|  85|     1056337.0|
|  84|     1117233.0|
|  83|     1033966.0|
|  82|     1063498.0|
|  81|     1053473.0|
+----+--------------+
only showing top 20 rows

--- 0.45450782775878906 seconds ---


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.partitionBy("date_built").mode("overwrite").parquet("hs_partitioned")

In [28]:
# 11. Read the parquet formatted data.

p_df=spark.read.parquet('hs_partitioned')
p_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [29]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView('hs_partitioned')

In [33]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.


start_time = time.time()
spark.sql("""

SELECT view, ROUND(AVG(price)) AS AVG_SALE_PRICE FROM hs_partitioned
GROUP BY view
HAVING AVG_SALE_PRICE >= 350000
ORDER BY view DESC

""").show()

print("--- %s seconds ---" % (time.time() - start_time))
#Runtime is fast but I am showing that it is a little bit slower than
# the cached temp table for this dataset

+----+--------------+
|view|AVG_SALE_PRICE|
+----+--------------+
| 100|     1026670.0|
|  99|     1061201.0|
|  98|     1053739.0|
|  97|     1129040.0|
|  96|     1017816.0|
|  95|     1054326.0|
|  94|     1033536.0|
|  93|     1026006.0|
|  92|      970403.0|
|  91|     1137373.0|
|  90|     1062654.0|
|  89|     1107839.0|
|  88|     1031719.0|
|  87|     1072285.0|
|  86|     1070444.0|
|  85|     1056337.0|
|  84|     1117233.0|
|  83|     1033966.0|
|  82|     1063498.0|
|  81|     1053473.0|
+----+--------------+
only showing top 20 rows

--- 0.5593500137329102 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")



In [35]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False